In [1]:
import pandas as pd
import numpy as np
import os
notebook_home = os.path.abspath('')
lookup_results = notebook_home + "/lookup_results"
test_results = notebook_home + "/test_results"
enron_stat = test_results + "/enron"
github_stat = test_results + "/github"
enron_output = lookup_results + "/enron/output"
enron_input = lookup_results + "/enron/input"
github_input = lookup_results + "/github/input"
github_output = lookup_results + "/github/output"

In [2]:
import seaborn as sns 
import matplotlib.pyplot as plt 

# 1. Compressed Formula Graph Sizes

In [79]:
def graph_compare_preprocess(df, is_inrow=False):
    df['Edges_Diff'] = df['numEdges'] - df['numCompEdges']
    df['Edges_Prec'] = df['numCompEdges'] / df['numEdges']
    if not is_inrow:
        pattern_type = ['RR', 'RF', 'FR', 'FF', 'RR-Chain', 'RRGap']
        for pattern in pattern_type:
            if pattern != 'RRGap':
                df[pattern + "_Diff"] = df[pattern + "_NoComp"] - df[pattern + "_Comp"]
            else:
                df[pattern + "_Diff"] = 0
                gaps = ['One', 'Two', 'Three', 'Four', 'Five', 'Six', 'Seven']
                for gap in gaps:
                    pattern_name = pattern + gap
                    df[pattern + "_Diff"] += (df[pattern_name + "_NoComp"] - df[pattern_name + "_Comp"])
    return df

In [80]:
df_enron_stat = graph_compare_preprocess(pd.read_csv(enron_stat + "/" + "enron_taco_stat_10k.csv"))
df_github_stat = graph_compare_preprocess(pd.read_csv(github_stat + "/" + "github_taco_stat_10k.csv"))
df_enron_inrow_stat = graph_compare_preprocess(pd.read_csv(enron_stat + "/" + "enron_inrow_taco_stat_10k.csv"), True)
df_github_inrow_stat = graph_compare_preprocess(pd.read_csv(github_stat + "/" + "github_inrow_taco_stat_10k.csv"), True)

In [81]:
df_enron_stat.head()

,fileName,numFormulae,numVertices,numEdges,numCompVertices,numCompEdges,RR-Chain_Comp,RR-Chain_NoComp,RR_Comp,RR_NoComp,...,NoComp_Comp,NoComp_NoComp,Edges_Diff,Edges_Prec,RR_Diff,RF_Diff,FR_Diff,FF_Diff,RR-Chain_Diff,RRGap_Diff
0,chris_germany_000_1_2.pst.1111.xls,8270,27200,26350,11520,11160,0,0,10230,24180,...,0,0,15190,0.423529,13950,0,0,0,0,1240
1,chris_germany_000_1_2.pst.1136.xls,1707,54464,52762,23059,22322,0,0,20446,48393,...,3,3,30440,0.423070,27947,0,0,0,0,2493
2,kevin_ruscitti_000_1_1.pst.100.xls,8268,27200,26350,11520,11160,0,0,10230,24180,...,0,0,15190,0.423529,13950,0,0,0,0,1240
3,kevin_ruscitti_000_1_1.pst.101.xls,8268,27200,26350,11520,11160,0,0,10230,24180,...,0,0,15190,0.423529,13950,0,0,0,0,1240
4,kevin_ruscitti_000_1_1.pst.152.xls,12398,27200,26350,11520,11160,0,0,10230,24180,...,0,0,15190,0.423529,13950,0,0,0,0,1240


## 1.1 Formula graph sizes after TACO compression (low is better)

In [82]:
df_graph_enron_compare = pd.DataFrame()
df_graph_enron_compare.index = ['NoComp', 'TACO-Inrow', 'TACO']
df_graph_enron_compare['Vertices'] = [df_enron_stat['numVertices'].sum(), df_enron_inrow_stat['numCompVertices'].sum(), df_enron_stat['numCompVertices'].sum()]
df_graph_enron_compare['Edges'] = [df_enron_stat['numEdges'].sum(), df_enron_inrow_stat['numCompEdges'].sum(), df_enron_stat['numCompEdges'].sum()]
df_graph_enron_compare['Vertices_Percentage'] = df_graph_enron_compare['Vertices'] / df_graph_enron_compare['Vertices'].max()
df_graph_enron_compare['Edges_Percentage'] = df_graph_enron_compare['Edges'] / df_graph_enron_compare['Edges'].max()
df_graph_enron_compare

,Vertices,Edges,Vertices_Percentage,Edges_Percentage
NoComp,18467065,27083750,1.000000,1.000000
TACO-Inrow,7501502,14077044,0.406210,0.519760
TACO,1315183,1210209,0.071218,0.044684


In [83]:
df_graph_github_compare = pd.DataFrame()
df_graph_github_compare.index = ['NoComp', 'TACO-Inrow', 'TACO']
df_graph_github_compare['Vertices'] = [df_github_stat['numVertices'].sum(), df_github_inrow_stat['numCompVertices'].sum(), df_github_stat['numCompVertices'].sum()]
df_graph_github_compare['Edges'] = [df_github_stat['numEdges'].sum(), df_github_inrow_stat['numCompVertices'].sum(), df_github_stat['numCompEdges'].sum()]
df_graph_github_compare['Vertices_Percentage'] = df_graph_github_compare['Vertices'] / df_graph_github_compare['Vertices'].max()
df_graph_github_compare['Edges_Percentage'] = df_graph_github_compare['Edges'] / df_graph_github_compare['Edges'].max()
df_graph_github_compare

,Vertices,Edges,Vertices_Percentage,Edges_Percentage
NoComp,160240056,216050038,1.000000,1.000000
TACO-Inrow,45453962,45453962,0.283662,0.210386
TACO,3524109,2884193,0.021993,0.013350


## 1.2 The number of edges reduced by TACO (high is better)

In [84]:
df_enron_edge_reduce = pd.DataFrame()
df_enron_edge_reduce['TACO-Inrow'] = df_enron_inrow_stat['Edges_Diff']
df_enron_edge_reduce['TACO'] = df_enron_stat['Edges_Diff']
df_enron_edge_reduce.describe().T[['max', '75%', '50%', 'mean']]

,max,75%,50%,mean
TACO-Inrow,383012.0,22108.0,15003.0,21933.736931
TACO,573599.0,43203.0,20417.0,43631.603710


In [86]:
df_github_edge_reduce = pd.DataFrame()
df_github_edge_reduce['TACO-Inrow'] = df_github_inrow_stat['Edges_Diff']
df_github_edge_reduce['TACO'] = df_github_stat['Edges_Diff']
df_github_edge_reduce.describe().T[['max', '75%', '50%', 'mean']]

,max,75%,50%,mean
TACO-Inrow,2193476.0,53279.5,22758.0,58334.711002
TACO,2455807.0,91166.5,42150.0,105404.228243


## 1.3 Remaining edge fractions after compression (low is better)

In [87]:
df_enron_edge_prec = pd.DataFrame()
df_enron_edge_prec['TACO-Inrow'] = df_enron_inrow_stat['Edges_Prec']
df_enron_edge_prec['TACO'] = df_enron_stat['Edges_Prec']
df_enron_edge_prec.describe().T[['min', '25%', '50%', 'mean']]

,min,25%,50%,mean
TACO-Inrow,0.00001,0.050397,0.377372,0.420695
TACO,0.00001,0.003395,0.013835,0.069730


In [88]:
df_github_edge_prec = pd.DataFrame()
df_github_edge_prec['TACO-Inrow'] = df_github_inrow_stat['Edges_Prec']
df_github_edge_prec['TACO'] = df_github_stat['Edges_Prec']
df_github_edge_prec.describe().T[['min', '25%', '50%', 'mean']]

,min,25%,50%,mean
TACO-Inrow,0.000005,0.000911,0.135342,0.358910
TACO,0.000002,0.000388,0.002020,0.025021


## 1.4 Number of edges reduced by each pattern (high is better)

In [89]:
df_enron_pattern_reduce = pd.DataFrame()
pattern_type = ['RR', 'RF', 'FR', 'FF', 'RR-Chain', 'RRGap']
for pattern in pattern_type:
    df_enron_pattern_reduce[pattern] = df_enron_stat[pattern + "_Diff"]
df_enron_pattern_reduce.describe().T[['mean', 'max']]

,mean,max
RR,27514.598651,258333.0
RF,3.168634,1413.0
FR,252.571669,13815.0
FF,6167.296796,170510.0
RR-Chain,759.703204,24596.0
RRGap,1163.087690,48670.0


In [90]:
df_github_pattern_reduce = pd.DataFrame()
pattern_type = ['RR', 'RF', 'FR', 'FF', 'RR-Chain', 'RRGap']
for pattern in pattern_type:
    df_github_pattern_reduce[pattern] = df_github_stat[pattern + "_Diff"]
df_github_pattern_reduce.describe().T[['mean', 'max']]

,mean,max
RR,61410.154541,1754081.0
RF,6.035698,9999.0
FR,80.550836,39008.0
FF,10051.552192,736581.0
RR-Chain,2610.785359,399996.0
RRGap,643.768188,572425.0


# 2. Building Graphs

In [91]:
column_list = ['fileName', 'GraphBuildTime']
df_enron_taco_max = pd.read_csv(f"{enron_output}/enron_max_taco_10k.csv")[column_list]
df_enron_taco_long = pd.read_csv(f"{enron_output}/enron_long_taco_10k.csv")[column_list]
df_enron_nocomp_max = pd.read_csv(f"{enron_output}/enron_max_nocomp_10k.csv")[column_list]
df_enron_nocomp_long = pd.read_csv(f"{enron_output}/enron_long_nocomp_10k.csv")[column_list]
df_github_taco_max =  pd.read_csv(f"{github_output}/github_max_taco_10k.csv")[column_list]
df_github_taco_long = pd.read_csv(f"{github_output}/github_long_taco_10k.csv")[column_list]
df_github_nocomp_max = pd.read_csv(f"{github_output}/github_max_nocomp_10k.csv")[column_list]
df_github_nocomp_long = pd.read_csv(f"{github_output}/github_long_nocomp_10k.csv")[column_list]

In [92]:
df_enron_taco = pd.merge(df_enron_taco_max, df_enron_taco_long, how="inner", on=["fileName"])
df_enron_nocomp = pd.merge(df_enron_nocomp_max, df_enron_nocomp_long, how="inner", on=["fileName"])
df_enron_taco['TACO_GraphBuildTime'] = (df_enron_taco['GraphBuildTime_x'] + df_enron_taco['GraphBuildTime_y']) / 2.
df_enron_nocomp['NoComp_GraphBuildTime'] = (df_enron_nocomp['GraphBuildTime_x'] + df_enron_nocomp['GraphBuildTime_y']) / 2.

In [93]:
enron_graph_result = pd.merge(df_enron_taco, df_enron_nocomp, how="inner", on=['fileName'])[['fileName', 'TACO_GraphBuildTime', 'NoComp_GraphBuildTime']]

In [94]:
enron_graph_result.head()

,fileName,TACO_GraphBuildTime,NoComp_GraphBuildTime
0,stacey_white_000_1_1.pst.254.xls,1681.0,1078.0
1,darron_c_giron_002_1_1_1.pst.177.xls,864.5,261.5
2,stacey_white_000_1_1.pst.262.xls,2751.5,1505.0
3,mike_grigsby_000_1_1_1.pst.248.xls,128.0,131.5
4,kevin_ruscitti_000_1_1.pst.156.xls,4269.0,170.5


In [95]:
enron_graph_result['TACO_GraphBuildTime'].mean(), enron_graph_result['NoComp_GraphBuildTime'].mean()

(1011.316722972973, 322.6418918918919)

In [96]:
df_github_taco = pd.merge(df_github_taco_max, df_github_taco_long, how="inner", on=["fileName"])
df_github_nocomp = pd.merge(df_github_nocomp_max, df_github_nocomp_long, how="inner", on=["fileName"])
df_github_taco['TACO_GraphBuildTime'] = (df_github_taco['GraphBuildTime_x'] + df_github_taco['GraphBuildTime_y']) / 2.
df_github_nocomp['NoComp_GraphBuildTime'] = (df_github_nocomp['GraphBuildTime_x'] + df_github_nocomp['GraphBuildTime_y']) / 2.

In [97]:
github_graph_result = pd.merge(df_github_taco, df_github_nocomp, how="inner", on=['fileName'])[['fileName', 'TACO_GraphBuildTime', 'NoComp_GraphBuildTime']]

In [98]:
github_graph_result['TACO_GraphBuildTime'].mean(), github_graph_result['NoComp_GraphBuildTime'].mean()

(1480.2192743764172, 745.8630385487528)

# 3. Returning Control to Users

In [118]:
def rename_columns(df, is_long=False):
    rename_map = {
        "TACOLookupSize": "taco_size",
        "TACOLookupTime": "taco_time",
        "TACOPostProcessedLookupSize": "taco_post_size",
        "TACOPostProcessedLookupTime": "taco_post_time",
        "NoCompLookupSize": "nocomp_size",
        "NoCompLookupTime": "nocomp_time",
        "NoCompPostProcessedLookupSize": "nocomp_post_size",
        "NoCompPostProcessedLookupTime": "nocomp_post_time",
    }
    
    if is_long:
        for key in rename_map:
            rename_map[key] = "long_" + rename_map[key]
    else:
        for key in rename_map:
            rename_map[key] = "max_" + rename_map[key]
            
    df = df.rename(columns=rename_map)
    if "GraphBuildTime" in list(df.columns):
        df = df.drop(['GraphBuildTime'], axis=1)
    
    remain_cols = [x for x in list(df.columns) if "post" not in x]
    return df[remain_cols]

In [133]:
def read_and_average_files(dir_path, fileName, is_long=False):
    df = rename_columns(pd.read_csv(dir_path + "/" + fileName + ".csv"), is_long)
    df2 = rename_columns(pd.read_csv(dir_path + "/" + fileName + "_2.csv"), is_long)
    df3 = rename_columns(pd.read_csv(dir_path + "/" + fileName + "_3.csv"), is_long)
    column_list = list(df.columns)
    df_merged = pd.merge(df, df2, how="inner", on=[column_list[0], column_list[1]])
    df_merged = pd.merge(df_merged, df3, how="inner", on=[column_list[0], column_list[1]])
    list1, list2 = [], []
    for i in range(len(df_merged)):
        ls = [df_merged.loc[i, col_name] for col_name in list(df_merged.columns) if column_list[2] in col_name]
        ls2 = [df_merged.loc[i, col_name] for col_name in list(df_merged.columns) if column_list[3] in col_name]
        list1.append(np.mean(ls))
        list2.append(np.mean(ls2))
    df_merged[column_list[2]] = list1
    df_merged[column_list[3]] = list2
    return df_merged[column_list]

In [137]:
df_enron_taco_max = read_and_average_files(enron_output, "enron_max_taco_10k", False)
df_enron_taco_long = read_and_average_files(enron_output, "enron_long_taco_10k", True)
df_enron_nocomp_max = read_and_average_files(enron_output, "enron_max_nocomp_10k", False)
df_enron_nocomp_long = read_and_average_files(enron_output, "enron_long_nocomp_10k", True)
df_github_taco_max = read_and_average_files(github_output, "github_max_taco_10k", False)
df_github_taco_long = read_and_average_files(github_output, "github_long_taco_10k", True)
df_github_nocomp_max = read_and_average_files(github_output, "github_max_nocomp_10k", False)
df_github_nocomp_long = read_and_average_files(github_output, "github_long_nocomp_10k", True)

In [140]:
df_github_nocomp_long.head()

,fileName,Longest Dep Ref,long_nocomp_size,long_nocomp_time
0,1. Excel Homework.xlsx,Sheet1:J3248,1.0,32.0
1,2019-05-23-0936_empty_racir.xlsx,Measurements:C11,5610.0,318.0
2,NIRISS_SIAF.xlsx,DDC:C20,22.0,0.0
3,工作簿1-20181122170204822.xlsx,不良明细:CEA5,2.0,0.0
4,W22704-181.xlsx,Sheet1:D2,14196.0,151.0


In [141]:
df_enron_max = pd.merge(df_enron_taco_max, df_enron_nocomp_max, how="inner", on=["fileName", "Max Dep Ref"])
df_enron_long = pd.merge(df_enron_taco_long, df_enron_nocomp_long, how="inner", on=["fileName", "Longest Dep Ref"])

In [142]:
df_enron_merged = pd.merge(df_enron_max, df_enron_long, how="inner", on=["fileName"])
# df_enron_merged.to_csv("./enron_result_merged.csv", index=False)

In [143]:
df_enron_merged.head()

,fileName,Max Dep Ref,max_taco_size,max_taco_time,max_nocomp_size,max_nocomp_time,Longest Dep Ref,long_taco_size,long_taco_time,long_nocomp_size,long_nocomp_time
0,stacey_white_000_1_1.pst.254.xls,AlphaSort:F6836,3.0,9.333333,3.0,29.0,AlphaSort:F4400,3.0,16.000000,3.0,30.000000
1,darron_c_giron_002_1_1_1.pst.177.xls,Darron:E5,205.0,8.333333,2978.0,78.0,Darron:J4,205.0,6.000000,2978.0,90.000000
2,stacey_white_000_1_1.pst.262.xls,BRA Power:A6,28.0,0.666667,4385.0,463.0,BRA Power:A6,28.0,0.666667,4385.0,495.333333
3,mike_grigsby_000_1_1_1.pst.248.xls,Months:F4,4.0,0.000000,306.0,3.0,Months:F4,4.0,0.333333,306.0,2.000000
4,kevin_ruscitti_000_1_1.pst.156.xls,03-21-00AM:BT499,0.0,0.000000,0.0,0.0,03-21-00AM:X531,1.0,0.000000,1.0,0.000000


In [144]:
df_github_max = pd.merge(df_github_taco_max, df_github_nocomp_max, how="inner", on=["fileName", "Max Dep Ref"])
df_github_long = pd.merge(df_github_taco_long, df_github_nocomp_long, how="inner", on=["fileName", "Longest Dep Ref"])

In [145]:
df_github_merged = pd.merge(df_github_max, df_github_long, how="inner", on=["fileName"])
# df_github_merged.to_csv("./github_result_merged.csv", index=False)

In [146]:
df_github_merged.head()

,fileName,Max Dep Ref,max_taco_size,max_taco_time,max_nocomp_size,max_nocomp_time,Longest Dep Ref,long_taco_size,long_taco_time,long_nocomp_size,long_nocomp_time
0,1. Excel Homework.xlsx,Sheet1:E4030,2.0,8.666667,2.0,28.333333,Sheet1:J3248,1.0,9.000000,1.0,32.0
1,2019-05-23-0936_empty_racir.xlsx,Measurements:E13,20.0,1.333333,6600.0,238.000000,Measurements:C11,17.0,1.000000,5610.0,318.0
2,NIRISS_SIAF.xlsx,Calc:AE3,23.0,0.333333,690.0,17.333333,DDC:C20,6.0,0.333333,22.0,0.0
3,工作簿1-20181122170204822.xlsx,不良明细:CBJ5,1.0,0.000000,2.0,0.333333,不良明细:CEA5,1.0,0.333333,2.0,0.0
4,W22704-181.xlsx,Sheet1:E2,4.0,0.000000,14196.0,172.000000,Sheet1:D2,4.0,0.333333,14196.0,151.0


### Analyze Enron results

In [147]:
df_enron_merged

,fileName,Max Dep Ref,max_taco_size,max_taco_time,max_nocomp_size,max_nocomp_time,Longest Dep Ref,long_taco_size,long_taco_time,long_nocomp_size,long_nocomp_time
0,stacey_white_000_1_1.pst.254.xls,AlphaSort:F6836,3.0,9.333333,3.0,29.0,AlphaSort:F4400,3.0,16.000000,3.0,30.000000
1,darron_c_giron_002_1_1_1.pst.177.xls,Darron:E5,205.0,8.333333,2978.0,78.0,Darron:J4,205.0,6.000000,2978.0,90.000000
2,stacey_white_000_1_1.pst.262.xls,BRA Power:A6,28.0,0.666667,4385.0,463.0,BRA Power:A6,28.0,0.666667,4385.0,495.333333
3,mike_grigsby_000_1_1_1.pst.248.xls,Months:F4,4.0,0.000000,306.0,3.0,Months:F4,4.0,0.333333,306.0,2.000000
4,kevin_ruscitti_000_1_1.pst.156.xls,03-21-00AM:BT499,0.0,0.000000,0.0,0.0,03-21-00AM:X531,1.0,0.000000,1.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
587,don_baughman_000_1_1.pst.127.xls,Sheet1:C47,37.0,0.333333,37.0,0.0,Sheet1:C38,37.0,0.333333,37.0,0.000000
588,kevin_ruscitti_000_1_1.pst.4.xls,Swap:A8,29.0,0.333333,1159.0,13.0,Swap:A8,29.0,0.000000,1159.0,17.000000
589,jane_tholt_000_1_1.pst.198.xls,Summary:B185,57.0,0.333333,1485.0,121.0,Study 4a:A9,12.0,0.333333,82.0,0.666667
590,andy_zipper_000_1_1.pst.67.xls,DD-ENA:F500,4.0,0.000000,8.0,0.0,DD-EGL:P29,3.0,0.000000,3.0,0.000000


In [148]:
df_larger_than = df_enron_merged[(df_enron_merged["max_taco_time"] > df_enron_merged["max_nocomp_time"]) & (df_enron_merged["max_nocomp_time"] > 10)]
df_larger_than[["fileName", "max_taco_time", "max_nocomp_time"]]

,fileName,max_taco_time,max_nocomp_time
19,kevin_ruscitti_000_1_1.pst.125.xls,53.0,52.0


In [149]:
threshold=500
df_enron_merged[df_enron_merged["max_nocomp_time"] > threshold][["fileName", "max_nocomp_time"]]

,fileName,max_nocomp_time
87,dutch_quigley_000_1_1.pst.51.xls,10006.0
127,mike_grigsby_000_1_1_1.pst.106.xls,1116.0
144,matthew_lenhart_000_1_1.pst.23.xls,1639.0
173,stacey_white_000_1_1.pst.287.xls,513.0
197,matthew_lenhart_000_1_1.pst.20.xls,1498.0
220,mark_haedicke_000_1_2.pst.31.xls,966.0
267,harry_arora_000_1_1.pst.130.xls,10214.0
269,matthew_lenhart_000_1_1.pst.1.xls,538.0
280,vkaminski_001_1_2_1.pst.62.xls,550.0
301,mike_grigsby_000_1_1_1.pst.108.xls,1378.0


In [150]:
df_enron_merged[df_enron_merged["long_nocomp_time"] > threshold][["fileName", "long_nocomp_time"]]

,fileName,long_nocomp_time
87,dutch_quigley_000_1_1.pst.51.xls,9307.666667
220,mark_haedicke_000_1_2.pst.31.xls,934.333333
267,harry_arora_000_1_1.pst.130.xls,12022.333333
280,vkaminski_001_1_2_1.pst.62.xls,546.666667
375,jim_schwieger_000_1_1.pst.100.xls,9857.333333
526,john_griffith_000_1_1.pst.185.xls,6268.000000
563,jim_schwieger_000_1_1.pst.71.xls,9573.666667
576,kevin_presto_000_1_1.pst.16.xls,4728.666667
585,jim_schwieger_000_1_1.pst.106.xls,9209.000000


In [151]:
df_enron_merged[df_enron_merged["max_taco_time"] > threshold][["fileName", "max_taco_time"]]

,fileName,max_taco_time


In [152]:
df_enron_merged[df_enron_merged["long_taco_time"] > threshold][["fileName", "long_taco_time"]]

,fileName,long_taco_time


### Analyze github dataset

In [153]:
df_larger_than = df_github_merged[(df_github_merged["max_taco_time"] > df_github_merged["max_nocomp_time"]) & (df_github_merged["max_nocomp_time"] > 10)]
df_larger_than[["fileName", "max_taco_time", "max_nocomp_time"]]

,fileName,max_taco_time,max_nocomp_time


In [154]:
threshold=500
df_github_merged[df_github_merged["max_nocomp_time"] > threshold][["fileName", "max_nocomp_time"]]

,fileName,max_nocomp_time
10,Fact2Sheet_Corona_20200607.xlsx,4483.666667
65,Kraken_BTCEUR_1h.xlsx,549.666667
90,crime_v3.xlsx,714.333333
152,gaslines_model_public.xlsx,788242.000000
154,Tio Cash Master 1400 Find Ratio of Star-fam.xlsx,579.000000
...,...,...
2151,经济总表.xlsx,41303.333333
2166,최경묵_퍼셉트론엑셀.xlsx,1304.333333
2196,aero_test.xlsx,1255.333333
2199,Ventas 2.0 exceldiario.xlsx,2094.333333


In [155]:
df_github_merged[df_github_merged["long_nocomp_time"] > threshold][["fileName", "long_nocomp_time"]]

,fileName,long_nocomp_time
59,地方政府债券整理-hs.xlsx,588.333333
90,crime_v3.xlsx,797.666667
131,DMD_PRJ.xlsx,1028.000000
152,gaslines_model_public.xlsx,701094.666667
154,Tio Cash Master 1400 Find Ratio of Star-fam.xlsx,547.000000
...,...,...
2112,Detroit data (stata & Eviews outputs).xlsx,2334.666667
2132,homework7_ducharme_h.xlsx,653.333333
2166,최경묵_퍼셉트론엑셀.xlsx,1409.333333
2196,aero_test.xlsx,1204.333333


In [156]:
df_github_merged[df_github_merged["max_taco_time"] > threshold][["fileName", "max_taco_time"]]

,fileName,max_taco_time


In [157]:
df_github_merged[df_github_merged["long_taco_time"] > threshold][["fileName", "long_taco_time"]]

,fileName,long_taco_time
